In [1]:
pip install keras

  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tensorflow


     ------------------------------------ 455.9/455.9 MB 344.5 kB/s eta 0:00:00
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
     ---------------------------------------- 1.5/1.5 MB 1.1 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached flatbuffers-22.9.24-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached termcolor-2.0.1-py3-none-any.whl (5.4 kB)
     -------------------------------------- 895.9/895.9 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 2.0 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
# conda install -c conda-forge tensorflow

In [4]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [6]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [7]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='ReLU', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 3.6742 - accuracy: 0.6364
Epoch 2/50
1/1 [==============================] - 0s 11ms/step - loss: 3.6739 - accuracy: 0.6364
Epoch 3/50
1/1 [==============================] - 0s 43ms/step - loss: 3.6736 - accuracy: 0.6364
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: 3.6734 - accuracy: 0.6364
Epoch 5/50
1/1 [==============================] - 0s 44ms/step - loss: 3.6732 - accuracy: 0.6364
Epoch 6/50
1/1 [==============================] - 0s 37ms/step - loss: 3.6729 - accuracy: 0.6364
Epoch 7/50
1/1 [==============================] - 0s 26ms/step - loss: 3.6727 - accuracy: 0.6364
Epoch 8/50
1/1 [==============================] - 0s 11ms/step - loss: 3.6724 - accuracy: 0.6364
Epoch 9/50
1/1 [==============================] - 0s 35ms/step - loss: 3.6722 - accuracy: 0.6364
Epoch 10/50
1/1 [==============================] - 0s 6ms/step - loss: 3.6720 - accuracy: 0.6364
Epoch 11/50
1/1 [================

In [8]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 225ms/step - loss: 5.1209 - accuracy: 0.6667


[5.12089729309082, 0.6666666865348816]

In [9]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 83ms/step


array([[1.4033577 ],
       [1.1266806 ],
       [0.20300524],
       [1.2220864 ],
       [1.422439  ],
       [1.536926  ]], dtype=float32)

In [10]:
y_test


2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [15]:
coef, intercept = model.get_weights()

In [16]:
coef, intercept

(array([[0.9509528],
        [0.9507531]], dtype=float32),
 array([-0.04950048], dtype=float32))

In [17]:
def sigmoid(x):
        import math
        return 1 / (1 + math.exp(-x))
sigmoid(18)

0.9999999847700205

In [20]:
X_test

,age,affordibility
2,47,1
10,18,1
21,26,0
11,28,1
14,49,1
9,61,1


In [21]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

prediction_function(.47, 1)

0.7938352665015822

In [22]:
prediction_function(.18, 1)

0.7450576531505623

Now we start implementing gradient descent in plain python. Again the goal is to come up with same w1, w2 and bias that keras model calculated. We want to show how keras/tensorflow would have computed these values internally using gradient descent

First write couple of helper routines such as sigmoid and log_loss

In [24]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [25]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))


In [26]:
class myNN:
    def __init__(self):
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")
        
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

In [31]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=8000, loss_thresold=0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:50, w1:1.503319554173139, w2:1.108384790367645, bias:-1.2319047301235464, loss:0.5675865113475955
Epoch:100, w1:2.200713131760032, w2:1.2941584023238903, bias:-1.6607009122062801, loss:0.5390680417774752
Epoch:150, w1:2.8495727769689085, w2:1.3696895491572747, bias:-1.986105845859897, loss:0.5176462164249293
Epoch:200, w1:3.443016970881804, w2:1.4042218624465028, bias:-2.2571369883752723, loss:0.5005011269691375
Epoch:250, w1:3.982450494649577, w2:1.423912732932123, bias:-2.494377365971801, loss:0.48654089537617096
Epoch:300, w1:4.4721795220959155, w2:1.438787986553552, bias:-2.7073878119223735, loss:0.4750814640632793
Epoch:350, w1:4.917245868007634, w2:1.4525660781176122, bias:-2.901176333556766, loss:0.46561475306999006
Epoch:366, w1:5.051047623653049, w2:1.4569794548473887, bias:-2.9596534546250037, loss:0.46293944095888917
Final weights and bias: w1: 5.051047623653049, w2:

In [28]:
coef, intercept

(array([[0.9509528],
        [0.9507531]], dtype=float32),
 array([-0.04950048], dtype=float32))

In [32]:
X_test_scaled


,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [33]:
customModel.predict(X_test_scaled)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64

In [34]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 15ms/step


array([[0.7938353 ],
       [0.74505764],
       [0.54927635],
       [0.7626963 ],
       [0.79693055],
       [0.814774  ]], dtype=float32)